<a href="https://colab.research.google.com/github/ZaidAl2004/AER850_Project2/blob/main/AER_850_Project_2_Parts_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AER 850: Project 2 Parts 5
# By: Zaid Al-lababidi
# 501176747


# Initializing all relevent libraries
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras import layers
#Issue: can't import image data generator without including tensorflow. even
#even though keras was imported earlier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Did not end up utilizing this
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')
#Step 5 Evaluating Model Test Performance

#Loading the model saved
from tensorflow.keras.models import load_model
mdl = load_model("/content/drive/MyDrive/model1.keras")

test_dir  = '/content/drive/MyDrive/project_2_data/Data/test'

#Defining Test Data Image Generator
test_datagen = ImageDataGenerator(rescale=1./255)


#Loading the Test Data
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size = (image_height, image_width),
    batch_size = 1, #Evaluate 1 image at a time
    class_mode = "categorical",
    color_mode='rgb',
    )


#Defining all the class labels
class_labels = list(test_gen.class_indices.keys())


#Visualize the predictions for targeted different images
target_filenames = ["crack/test_crack.jpg",
                    'missing-head/test_missinghead.jpg',
                    'paint-off/test_paintoff.jpg']


for img_path in target_filenames:

    img_full_path = os.path.join(test_dir, img_path)
    img = image.load_img(img_full_path, target_size=(500, 500))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    #Predict Probabilities
    preds = mdl.predict(img_array) #Predicts what the img should be
    pred_class = class_labels[np.argmax(preds)] #retrieve the class the model predicts

    index = test_gen.filenames.index(img_path) #Retrieves the index of the image
    true_label_index = test_gen.classes[index] #Retrieves the index of the true label
    true_label = class_labels[true_label_index] #Retrievs the actual true label

    #plot the image + display the probabilities
    plt.imshow(img)
    plt.axis("off")
    plt.title(
        f"True Classification Label: {true_label}\n",
        f"Predicted Classification Label: {pred_class}",
        fontsize=10, loc='left', pad=10
        )

    text = '\n'.join([
        f"{cls.capitalize()}: {preds[j]*100:.1f}%" for j, cls in enumerate(class_labels)
        ])
    plt.text(10, 470, text, color='green', fontsize=10, weight='bold')
    plt.show()